# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units

baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units

baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer

baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model

baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model

baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))


Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [===============

33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 65/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 66/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [=================

33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
33/33 [==============================] - 0s 924us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [=========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names

numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler

ss_X = StandardScaler()


# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)


# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features

X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 

normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 849us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 849us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [===============

33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 865us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 894us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 863us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 864us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [=================

Epoch 135/150
33/33 [==============================] - 0s 849us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
33/33 [==============================] - 0s 849us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 848us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 880us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 893us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 895us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 863us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 818us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler

ss_y = StandardScaler()

# Fit and transform train labels

y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels

y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs

np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model

normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model

normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5473 - mse: 0.5473 - val_loss: 0.1753 - val_mse: 0.1753
Epoch 2/150
33/33 [==============================] - 0s 864us/step - loss: 0.2396 - mse: 0.2396 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 3/150
33/33 [==============================] - 0s 879us/step - loss: 0.1955 - mse: 0.1955 - val_loss: 0.1408 - val_mse: 0.1408
Epoch 4/150
33/33 [==============================] - 0s 833us/step - loss: 0.1703 - mse: 0.1703 - val_loss: 0.1380 - val_mse: 0.1380
Epoch 5/150
33/33 [==============================] - 0s 895us/step - loss: 0.1546 - mse: 0.1546 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 6/150
33/33 [==============================] - 0s 849us/step - loss: 0.1447 - mse: 0.1447 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 7/150
33/33 [==============================] - 0s 848us/step - loss: 0.1282 - mse: 0.1282 - val_loss: 0.1376 - val_mse: 0.1376
Epoch 8/150
33/33 [==============================] - 0s 849us/step - lo

33/33 [==============================] - 0s 894us/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 63/150
33/33 [==============================] - 0s 833us/step - loss: 0.0225 - mse: 0.0225 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 64/150
33/33 [==============================] - 0s 864us/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 65/150
33/33 [==============================] - 0s 879us/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1265 - val_mse: 0.1265
Epoch 66/150
33/33 [==============================] - 0s 848us/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1262 - val_mse: 0.1262
Epoch 67/150
33/33 [==============================] - 0s 864us/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1271 - val_mse: 0.1271
Epoch 68/150
33/33 [==============================] - 0s 879us/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 69/150
33/33 [==============================] - 0s 894us/step - loss:

33/33 [==============================] - 0s 879us/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 124/150
33/33 [==============================] - 0s 864us/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1254 - val_mse: 0.1254
Epoch 125/150
33/33 [==============================] - 0s 848us/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 126/150
33/33 [==============================] - 0s 970us/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 127/150
33/33 [==============================] - 0s 833us/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1264 - val_mse: 0.1264
Epoch 128/150
33/33 [==============================] - 0s 894us/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1255 - val_mse: 0.1255
Epoch 129/150
33/33 [==============================] - 0s 864us/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1253 - val_mse: 0.1253
Epoch 130/150
33/33 [==============================] - 0s 833us/step 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data

normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 486us/step - loss: 0.0075 - mse: 0.0075


[0.007502123713493347, 0.007502123713493347]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data

normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 666us/step - loss: 0.1269 - mse: 0.1269


[0.12688185274600983, 0.12688185274600983]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data

y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale

y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data

np.sqrt(mean_squared_error(y_val, y_val_pred))

27991.829805153586

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer

he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer

he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer

he_model.add(layers.Dense(1, activation='linear'))

# Compile the model

he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model

he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5670 - mse: 0.5670 - val_loss: 0.2411 - val_mse: 0.2411
Epoch 2/150
33/33 [==============================] - 0s 894us/step - loss: 0.2838 - mse: 0.2838 - val_loss: 0.1943 - val_mse: 0.1943
Epoch 3/150
33/33 [==============================] - 0s 894us/step - loss: 0.2217 - mse: 0.2217 - val_loss: 0.1710 - val_mse: 0.1710
Epoch 4/150
33/33 [==============================] - 0s 879us/step - loss: 0.2003 - mse: 0.2003 - val_loss: 0.1678 - val_mse: 0.1678
Epoch 5/150
33/33 [==============================] - 0s 864us/step - loss: 0.1702 - mse: 0.1702 - val_loss: 0.1438 - val_mse: 0.1438
Epoch 6/150
33/33 [==============================] - 0s 833us/step - loss: 0.1610 - mse: 0.1610 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 7/150
33/33 [==============================] - 0s 833us/step - loss: 0.1445 - mse: 0.1445 - val_loss: 0.1426 - val_mse: 0.1426
Epoch 8/150
33/33 [==============================] - 0s 833us/step - lo

33/33 [==============================] - 0s 880us/step - loss: 0.0278 - mse: 0.0278 - val_loss: 0.1600 - val_mse: 0.1600
Epoch 63/150
33/33 [==============================] - 0s 865us/step - loss: 0.0270 - mse: 0.0270 - val_loss: 0.1565 - val_mse: 0.1565
Epoch 64/150
33/33 [==============================] - 0s 863us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1566 - val_mse: 0.1566
Epoch 65/150
33/33 [==============================] - 0s 880us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1586 - val_mse: 0.1586
Epoch 66/150
33/33 [==============================] - 0s 864us/step - loss: 0.0259 - mse: 0.0259 - val_loss: 0.1563 - val_mse: 0.1563
Epoch 67/150
33/33 [==============================] - 0s 879us/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1574 - val_mse: 0.1574
Epoch 68/150
33/33 [==============================] - 0s 879us/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1585 - val_mse: 0.1585
Epoch 69/150
33/33 [==============================] - 0s 879us/step - loss:

33/33 [==============================] - 0s 865us/step - loss: 0.0130 - mse: 0.0130 - val_loss: 0.1734 - val_mse: 0.1734
Epoch 124/150
33/33 [==============================] - 0s 879us/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1751 - val_mse: 0.1751
Epoch 125/150
33/33 [==============================] - 0s 867us/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1761 - val_mse: 0.1761
Epoch 126/150
33/33 [==============================] - 0s 909us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1750 - val_mse: 0.1750
Epoch 127/150
33/33 [==============================] - 0s 909us/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1732 - val_mse: 0.1732
Epoch 128/150
33/33 [==============================] - 0s 894us/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1750 - val_mse: 0.1750
Epoch 129/150
33/33 [==============================] - 0s 909us/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1792 - val_mse: 0.1792
Epoch 130/150
33/33 [==============================] - 0s 925us/step 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data

he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 440us/step - loss: 0.0102 - mse: 0.0102


[0.010236456990242004, 0.010236456990242004]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data

he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 503us/step - loss: 0.1843 - mse: 0.1843


[0.18433572351932526, 0.18433572351932526]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer

lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer

lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer

lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model

lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model

lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.4106 - mse: 0.4106 - val_loss: 0.1955 - val_mse: 0.1955
Epoch 2/150
33/33 [==============================] - 0s 879us/step - loss: 0.2225 - mse: 0.2225 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 3/150
33/33 [==============================] - 0s 879us/step - loss: 0.1850 - mse: 0.1850 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 4/150
33/33 [==============================] - 0s 848us/step - loss: 0.1534 - mse: 0.1534 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 5/150
33/33 [==============================] - 0s 849us/step - loss: 0.1357 - mse: 0.1357 - val_loss: 0.1338 - val_mse: 0.1338
Epoch 6/150
33/33 [==============================] - 0s 864us/step - loss: 0.1218 - mse: 0.1218 - val_loss: 0.1305 - val_mse: 0.1305
Epoch 7/150
33/33 [==============================] - 0s 864us/step - loss: 0.1127 - mse: 0.1127 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 8/150
33/33 [==============================] - 0s 849us/step - lo

33/33 [==============================] - 0s 895us/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 63/150
33/33 [==============================] - 0s 894us/step - loss: 0.0248 - mse: 0.0248 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 64/150
33/33 [==============================] - 0s 894us/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 65/150
33/33 [==============================] - 0s 880us/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 66/150
33/33 [==============================] - 0s 893us/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1305 - val_mse: 0.1305
Epoch 67/150
33/33 [==============================] - 0s 894us/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 68/150
33/33 [==============================] - 0s 864us/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 69/150
33/33 [==============================] - 0s 848us/step - loss:

33/33 [==============================] - 0s 864us/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1337 - val_mse: 0.1337
Epoch 124/150
33/33 [==============================] - 0s 864us/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 125/150
33/33 [==============================] - 0s 909us/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 126/150
33/33 [==============================] - 0s 848us/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1335 - val_mse: 0.1335
Epoch 127/150
33/33 [==============================] - 0s 909us/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1348 - val_mse: 0.1348
Epoch 128/150
33/33 [==============================] - 0s 894us/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 129/150
33/33 [==============================] - 0s 909us/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1343 - val_mse: 0.1343
Epoch 130/150
33/33 [==============================] - 0s 909us/step 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data

lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 515us/step - loss: 0.0087 - mse: 0.0087


[0.008706280961632729, 0.008706280961632729]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data

lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 558us/step - loss: 0.1357 - mse: 0.1357


[0.13574622571468353, 0.13574622571468353]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model

rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model

rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.3975 - mse: 0.3975 - val_loss: 0.1631 - val_mse: 0.1631
Epoch 2/150
33/33 [==============================] - 0s 924us/step - loss: 0.1940 - mse: 0.1940 - val_loss: 0.1336 - val_mse: 0.1336
Epoch 3/150
33/33 [==============================] - 0s 924us/step - loss: 0.1487 - mse: 0.1487 - val_loss: 0.1502 - val_mse: 0.1502
Epoch 4/150
33/33 [==============================] - 0s 924us/step - loss: 0.1129 - mse: 0.1129 - val_loss: 0.1557 - val_mse: 0.1557
Epoch 5/150
33/33 [==============================] - 0s 909us/step - loss: 0.0977 - mse: 0.0977 - val_loss: 0.1246 - val_mse: 0.1246
Epoch 6/150
33/33 [==============================] - 0s 909us/step - loss: 0.0814 - mse: 0.0814 - val_loss: 0.2013 - val_mse: 0.2013
Epoch 7/150
33/33 [==============================] - 0s 939us/step - loss: 0.0718 - mse: 0.0718 - val_loss: 0.1357 - val_mse: 0.1357
Epoch 8/150
33/33 [==============================] - 0s 909us/step - lo

33/33 [==============================] - 0s 910us/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 63/150
33/33 [==============================] - 0s 894us/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1549 - val_mse: 0.1549
Epoch 64/150
33/33 [==============================] - 0s 894us/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1691 - val_mse: 0.1691
Epoch 65/150
33/33 [==============================] - 0s 894us/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1460 - val_mse: 0.1460
Epoch 66/150
33/33 [==============================] - 0s 909us/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 67/150
33/33 [==============================] - 0s 895us/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 68/150
33/33 [==============================] - 0s 909us/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.1361 - val_mse: 0.1361
Epoch 69/150
33/33 [==============================] - 0s 894us/step - loss:

33/33 [==============================] - 0s 940us/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 124/150
33/33 [==============================] - 0s 939us/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 125/150
33/33 [==============================] - 0s 924us/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.1235 - val_mse: 0.1235
Epoch 126/150
33/33 [==============================] - 0s 894us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1327 - val_mse: 0.1327
Epoch 127/150
33/33 [==============================] - 0s 894us/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 128/150
33/33 [==============================] - 0s 910us/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 129/150
33/33 [==============================] - 0s 909us/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1465 - val_mse: 0.1465
Epoch 130/150
33/33 [==============================] - 0s 909us/step 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data

rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 440us/step - loss: 0.0126 - mse: 0.0126


[0.012560183182358742, 0.012560183182358742]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on validate data

rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 556us/step - loss: 0.1409 - mse: 0.1409


[0.14090567827224731, 0.14090567827224731]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model

adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model

adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4733 - mse: 0.4733 - val_loss: 0.1636 - val_mse: 0.1636
Epoch 2/150
33/33 [==============================] - 0s 939us/step - loss: 0.1740 - mse: 0.1740 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 3/150
33/33 [==============================] - 0s 944us/step - loss: 0.1231 - mse: 0.1231 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 4/150
33/33 [==============================] - 0s 950us/step - loss: 0.0930 - mse: 0.0930 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 5/150
33/33 [==============================] - 0s 917us/step - loss: 0.0744 - mse: 0.0744 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 6/150
33/33 [==============================] - 0s 916us/step - loss: 0.0559 - mse: 0.0559 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 7/150
33/33 [==============================] - 0s 935us/step - loss: 0.0506 - mse: 0.0506 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 8/150
33/33 [==============================] - 0s 900us/step - lo

33/33 [==============================] - 0s 924us/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 63/150
33/33 [==============================] - 0s 880us/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 64/150
33/33 [==============================] - 0s 909us/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 65/150
33/33 [==============================] - 0s 910us/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 66/150
33/33 [==============================] - 0s 909us/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 67/150
33/33 [==============================] - 0s 924us/step - loss: 9.6420e-04 - mse: 9.6420e-04 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 68/150
33/33 [==============================] - 0s 909us/step - loss: 9.5679e-04 - mse: 9.5679e-04 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 69/150
33/33 [==============================] - 0s 93

33/33 [==============================] - 0s 909us/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.0973 - val_mse: 0.0973
Epoch 123/150
33/33 [==============================] - 0s 939us/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.0954 - val_mse: 0.0954
Epoch 124/150
33/33 [==============================] - 0s 924us/step - loss: 8.8855e-04 - mse: 8.8855e-04 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 125/150
33/33 [==============================] - 0s 924us/step - loss: 7.1461e-04 - mse: 7.1461e-04 - val_loss: 0.0986 - val_mse: 0.0986
Epoch 126/150
33/33 [==============================] - 0s 909us/step - loss: 7.3407e-04 - mse: 7.3407e-04 - val_loss: 0.0957 - val_mse: 0.0957
Epoch 127/150
33/33 [==============================] - 0s 909us/step - loss: 7.7010e-04 - mse: 7.7010e-04 - val_loss: 0.0968 - val_mse: 0.0968
Epoch 128/150
33/33 [==============================] - 0s 909us/step - loss: 7.5257e-04 - mse: 7.5257e-04 - val_loss: 0.0982 - val_mse: 0.0982
Epoch 129/150
33/33 [========

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data

adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 470us/step - loss: 3.4064e-04 - mse: 3.4064e-04


[0.0003406406904105097, 0.0003406406904105097]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data

adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 558us/step - loss: 0.0929 - mse: 0.0929


[0.09285809099674225, 0.09285809099674225]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [27]:
# Evaluate the best model on test data

rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 600us/step - loss: 0.2027 - mse: 0.2027


[0.20267243683338165, 0.20267243683338165]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [28]:
# Generate predictions on test data

y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale

y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data

np.sqrt(mean_squared_error(y_test, y_test_pred))

35377.63584109672

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.